In [110]:
import PyPDF2
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
import requests 

In [131]:
PDFfilename = "estreallas.pdf"
fileData = (PDFfilename, open(PDFfilename, 'rb')) #"rb" stands for "read bytes"

files = {'f': fileData} 

apiKey = "uiv2c4isbi5o" 

fileExt = "csv" #format/file extension of final document

postUrl = "https://pdftables.com/api?key={0}&format={1}".format(apiKey, fileExt)
#the .format puts value of apiKey where {0} is, etc

response = requests.post(postUrl, files=files)

response.raise_for_status() # ensure we notice bad responses

downloadDir = "stars.csv" #directory where you want your file downloaded to 

with open(downloadDir, "wb") as f:
    f.write(response.content) #write data to csv
print(downloadDir)

stars.csv


In [135]:
file_name = 'stars2.csv'
data = pd.read_csv(file_name)
data

BRIGHT STARS,   Ascension Declination
0              9072  00 00 15.8   C06 57 57
1              9076  00 00 51.9    65 28 27
2              9084  00 02 31.3    76 57 49
3              9089  00 02 54.5    05 54 41
4              9098  00 04 41.2    17 13 59
5                 3  00 06 17.0    05 36 15
6                15  00 09 20.9   C29 11 33
7                21  00 10 10.6   C59 15 06
8                25  00 10 20.7    45 38 44
9                27  00 11 17.4   C46 10 31
10               34  00 12 30.6    27 41 49
11               35  00 12 40.2    35 01 47
12               39  00 14 11.4   C15 17 11
13               45  00 15 33.8   C20 18 34
14               48  00 15 34.7    18 49 50
15               68  00 19 18.0   C36 53 16
16               74  00 20 22.2    08 43 17
17               77  00 21 01.4    64 45 58
18               80  00 21 33.1   C08 17 34
19               82  00 22 06.1   C38 04 15
20               90  00 25 00.9   C38 40 45
21               98  00 26 41.9    77 09 01
22              100  00 27 06.5    43 34 39
23               99  00 27 11.7    42 12 20
24              118  00 31 18.0    23 41 08
25              125  00 32 18.2    48 42 05
26              126  00 32 23.1    62 51 23
27              130  00 34 04.0   C63 02 01
28              154  00 37 52.4   C33 49 15
29              153  00 38 00.7   C53 59 54
...             ...         ...         ...
1439           8905  23 26 18.3   C23 30 22
1440           8906  23 27 01.0    20 32 26
1441           8911  23 27 52.9   C01 21 25
1442           8916  23 28 54.4   C06 28 51
1443           8923  23 30 05.5   C12 51 46
1444           8862  23 30 15.0    87 22 48
1445           8924  23 30 29.4    04 25 55
1446           8937  23 33 57.5    37 42 57
1447           8952  23 35 47.9   C71 44 40
1448           8949  23 36 03.9    42 30 45
1449           8961  23 38 28.5   C46 33 31
1450           8959  23 38 50.4    45 23 24
1451           8965  23 39 03.0   C43 22 14
1452           8974  23 40 07.5   C77 44 08
1453           8969  23 40 54.2   C05 43 36
1454           8976  23 41 19.5   C44 26 11
1455           8975  23 41 36.2    31 58 15
1456           8984  23 42 59.5   C01 52 55
1457           8988  23 43 40.8    14 26 33
1458           8998  23 45 09.5    18 10 27
1459           9003  23 46 57.4   C46 31 23
1460           9013  23 48 48.6   C67 54 35
1461           9012  23 48 53.6    02 39 31
1462           9016  23 49 53.2    28 01 41
1463           9036  23 53 25.9   C19 13 23
1464           9039  23 53 33.8   C11 03 01
1465           9045  23 55 19.0   C57 36 08
1466           9064  23 58 42.3   C25 14 39
1467           9067  23 59 37.2    03 27 12
1468           9069  23 59 52.9    52 38 33

[1469 rows x 3 columns]

In [136]:
brillos = data.iloc[:,0].values
asc = data.iloc[:,1].values
desc = data.iloc[:,2].values


In [137]:
RA_h = []
RA_m = []
RA_s = []
for d in asc:
    RA_h.append(float(str(d).split(' ')[0])*15)
    RA_m.append(float(str(d).split(' ')[1])*.25)
    RA_s.append(float(str(d).split(' ')[2])*0.004166)
RA_h = np.array(RA_h)
RA_m = np.array(RA_m)
RA_s = np.array(RA_s)
A = RA_h + RA_m + RA_s
A

array([6.58228000e-02, 2.16215400e-01, 6.30395800e-01, ...,
       3.59676222e+02, 3.59904975e+02, 3.59970381e+02])

In [138]:
D_h = []
D_m = []
D_s = []
for r in desc:
    h = str(r).split(' ')[0]
    m = h.split('C')
    if(len(m) == 2):
        insertion = float(m[1])
    else:
        insertion = -1*float(m[0])
    D_h.append(insertion) 
    D_m.append(float(str(r).split(' ')[1])) 
    D_s.append(float(str(r).split(' ')[2])) 
D_h = np.array(D_h)
D_m = np.array(D_m)
D_s = np.array(D_s)
B = (abs(D_h) + (D_m/60) + D_s/3600)*np.sign(D_h)
B

array([  6.96583333, -65.47416667, -76.96361111, ...,  25.24416667,
        -3.45333333, -52.6425    ])

In [183]:
C = np.linspace(100, 3010, len(B))
C

array([ 100.        ,  101.98228883,  103.96457766, ..., 3006.03542234,
       3008.01771117, 3010.        ])

In [184]:
x = (C*np.cos(B))*np.cos(A)
x

array([ 7.74224682e+01, -8.74484483e+01,  4.54248121e-01, ...,
        1.07932063e+02,  5.48063635e+02,  5.54290106e+02])

In [185]:
y = (C*np.cos(B))*np.sin(A)
y

array([ 5.10353637e+00, -1.92079565e+01,  3.31474445e-01, ...,
        2.98544343e+03, -2.81008781e+03, -2.10033733e+03])

In [186]:
z = C*np.sin(B)
z

array([   63.08498499,   -48.82837833,  -103.96305686, ...,
         334.25614357,   922.60680295, -2083.61359024])

In [205]:
fig = plt.figure(figsize=(15,15))
#ax = fig.gca(projection='3d')
ax = plt.axes(projection='3d')
ax.set_facecolor("gray")
ax.plot([0], [0], [0], label="Sol", color='green', marker='*',)
ax.scatter3D(x, y, z, c=z, cmap='Reds')
plt.show()
print(3)

3


In [98]:
fileName = "estrellas.csv"
data = pd.read_csv(fileName)
data

Stellar   Mass    Luminosity        Radius    Temp  Color Index  Abs Mag  \
0        DA0    1.1  6.910000e+00       0.00902  100000        -0.37     10.2   
1        DB0    1.1  6.910000e+00       0.00902  100000        -0.37     10.2   
2        DC0    1.1  6.910000e+00       0.00902  100000        -0.37     10.2   
3        DO0    1.1  6.910000e+00       0.00902  100000        -0.37     10.2   
4        DQ0    1.1  6.910000e+00       0.00902  100000        -0.37     10.2   
5        DZ0    1.1  6.910000e+00       0.00902  100000        -0.37     10.2   
6        DA1    0.9  2.650000e-01       0.00696   50400        -0.35     10.8   
7        DB1    0.9  2.650000e-01       0.00696   50400        -0.35     10.8   
8        DC1    0.9  2.650000e-01       0.00696   50400        -0.35     10.8   
9        DO1    0.9  2.650000e-01       0.00696   50400        -0.35     10.8   
10       DQ1    0.9  2.650000e-01       0.00696   50400        -0.35     10.8   
11       DZ1    0.9  2.650000e-01       0.00696   50400        -0.35     10.8   
12     O0Ia0  160.0  3.410000e+07      80.20000   50000        -0.35     -9.5   
13    WC0Ia0  160.0  3.410000e+07      80.20000   50000        -0.35     -9.5   
14    WN0Ia0  160.0  3.410000e+07      80.20000   50000        -0.35     -9.5   
15      O0Ia  150.0  2.590000e+06      22.10000   50000        -0.35     -6.7   
16     WC0Ia  150.0  2.590000e+06      22.10000   50000        -0.35     -6.7   
17     WN0Ia  150.0  2.590000e+06      22.10000   50000        -0.35     -6.7   
18      O0Ib  140.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
19     WC0Ib  140.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
20     WN0Ib  140.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
21      O0II  130.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
22     WC0II  130.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
23     WN0II  130.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
24     O0III  120.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
25    WC0III  120.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
26    WN0III  120.0  2.150000e+06      20.20000   50000        -0.35     -6.5   
27      O0IV  110.0  1.360000e+06      16.00000   50000        -0.35     -6.0   
28     WC0IV  110.0  1.360000e+06      16.00000   50000        -0.35     -6.0   
29     WN0IV  110.0  1.360000e+06      16.00000   50000        -0.35     -6.0   
...      ...    ...           ...           ...     ...          ...      ...   
1070    N9IV    2.1  7.910000e+03     764.00000    2000         2.39      3.3   
1071    S9IV    2.1  7.910000e+03     764.00000    2000         2.39      3.3   
1072     C9V    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1073    C9VI    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1074     M9V    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1075    M9VI    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1076     N9V    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1077    N9VI    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1078     S9V    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1079    S9VI    0.1  4.150000e-03       0.55300    2000         2.39     19.0   
1080   C9Ia0   12.0  7.110000e+08  231000.00000    1990         2.40     -9.0   
1081   M9Ia0   12.0  7.110000e+08  231000.00000    1990         2.40     -9.0   
1082   N9Ia0   12.0  7.110000e+08  231000.00000    1990         2.40     -9.0   
1083   S9Ia0   12.0  7.110000e+08  231000.00000    1990         2.40     -9.0   
1084    C9Ia   10.0  1.030000e+08   87900.00000    1990         2.40     -6.9   
1085    M9Ia   10.0  1.030000e+08   87900.00000    1990         2.40     -6.9   
1086    N9Ia   10.0  1.030000e+08   87900.00000    1990         2.40     -6.9   
1087    S9Ia   10.0  1.030000e+08   

In [100]:
x = np.array(data.iloc[:,4].values)
y = np.array(data.iloc[:,3].values)
z = np.array(data.iloc[:,2].values)

In [130]:
def obj_ridge(b, x, y, n, l):
    return np.sum((y-b.dot([x**i for i in range(n+1)]))**2)+l*np.linalg.norm(b)**2

b0 = np.random.random((8,))
res = opt.minimize(obj_ridge, b0, args=(x*y, z, 7, 0.0001))
yhat7_ridge = np.array([(x*y)**j for j in range(8)]).T.dot(res.x)

def obj_lasso(b, x, y, n, l):
    return np.sum((y-b.dot([x**i for i in range(n+1)]))**2)+l*np.linalg.norm(b,1)

b0 = np.random.random((8,))
res = opt.minimize(obj_lasso, b0, args=(x*y, z, 7, 0.0001))
yhat7_lasso = np.array([(x*y)**j for j in range(8)]).T.dot(res.x)



In [134]:
fig = plt.figure()
plt.scatter(x*y, z, label='Estrellas')
plt.plot(x*y, yhat7_ridge, "r.", label="Ajuste ridge")
plt.plot(x*y, yhat7_lasso, "b*", label="Ajuste lasso")
plt.legend(loc="best")
plt.show()